In [21]:
using LinearAlgebra
using Random
using Distributions

In [20]:
K = 3
X = [1 10.5;1.5 10.8; 1.8 8; 1.7 15; 3.2 40; 3.6 32; 3.3 38; 5.1 -2.3; 5.2 -2.4]

9×2 Array{Float64,2}:
 1.0  10.5
 1.5  10.8
 1.8   8.0
 1.7  15.0
 3.2  40.0
 3.6  32.0
 3.3  38.0
 5.1  -2.3
 5.2  -2.4

In [2]:
"""
  initRepresentatives(X,K;initStrategy,Z₀))

Initialisate the representatives for a K-Mean or K-Medoids algorithm

# Parameters:
* `X`: a (N x D) data to clusterise
* `K`: Number of cluster wonted
* `initStrategy`: Wheter to select the initial representative vectors:
  * `random`: randomly in the X space
  * `grid`: using a grid approach [default]
  * `shuffle`: selecting randomly within the available points
  * `given`: using a provided set of initial representatives provided in the `Z₀` parameter
 * `Z₀`: Provided (K x D) matrix of initial representatives (used only together with the `given` initStrategy) [default: `nothing`]

# Returns:
* A (K x D) matrix of initial representatives

# Example:
```julia
julia> Z₀ = initRepresentatives([1 10.5;1.5 10.8; 1.8 8; 1.7 15; 3.2 40; 3.6 32; 3.6 38],2,initStrategy="given",Z₀=[1.7 15; 3.6 40])
```
"""
function initRepresentatives(X,K;initStrategy="grid",Z₀=nothing)
    (N,D) = size(X)
    # Random choice of initial representative vectors (any point, not just in X!)
    minX = minimum(X,dims=1)
    maxX = maximum(X,dims=1)
    Z = zeros(K,D)
    if initStrategy == "random"
        for i in 1:K
            for j in 1:D
                Z[i,j] = rand(Uniform(minX[j],maxX[j]))
            end
        end
    elseif initStrategy == "grid"
        for d in 1:D
                Z[:,d] = collect(range(minX[d], stop=maxX[d], length=K))
        end
    elseif initStrategy == "given"
        if isnothing(Z₀) error("With the `given` strategy you need to provide the initial set of representatives in the Z₀ parameter.") end
        for d in 1:D
                Z = Z₀
        end
    elseif initStrategy == "shuffle"
        for d in 1:D
            zIdx = shuffle(1:size(X)[1])[1:K]
            Z = X[zIdx, :]
        end
    else
        error("initStrategy \"$initStrategy\" not implemented")
    end
    return Z
end


initRepresentatives

In [4]:
Z₀ = initRepresentatives([1 10.5;1.5 10.8; 1.8 8; 1.7 15; 3.2 40; 3.6 32; 3.6 38],2,initStrategy="grid")

2×2 Array{Float64,2}:
 1.0   8.0
 3.6  40.0

In [8]:
# Basic K-Means Algorithm (Lecture/segment 13.7 of https://www.edx.org/course/machine-learning-with-python-from-linear-models-to)

"""
  kmeans(X,K,initStrategy)

Compute K-Mean algorithm to identify K clusters of X using Euclidean distance

# Parameters:
* `X`: a (N x D) data to clusterise
* `K`: Number of cluster wonted
* `initStrategy`: Wheter to select the initial representative vectors:
  * `random`: randomly in the X space
  * `grid`: using a grid approach [default]
  * `shuffle`: selecting randomly within the available points
  * `given`: using a provided set of initial representatives provided in the `Z₀` parameter
 * `Z₀`: Provided (K x D) matrix of initial representatives (used only together with the `given` initStrategy) [default: `nothing`]

# Returns:
* A tuple of two items, the first one being a vector of size N of ids of the clusters associated to each point and the second one the (K x D) matrix of representatives

# Notes:
* Some returned clusters could be empty

# Example:
```julia
julia> (clIdx,Z) = kmeans([1 10.5;1.5 10.8; 1.8 8; 1.7 15; 3.2 40; 3.6 32; 3.3 38; 5.1 -2.3; 5.2 -2.4],3)
```
"""
function kmeans(X,K;initStrategy="grid",Z₀=nothing)
    (N,D) = size(X)
    # Random choice of initial representative vectors (any point, not just in X!)
    minX = minimum(X,dims=1)
    maxX = maximum(X,dims=1)
    Z₀ = initRepresentatives(X,K,initStrategy=initStrategy,Z₀=Z₀)
    Z  = Z₀
    cIdx_prev = zeros(Int64,N)

    # Looping
    while true
        # Determining the constituency of each cluster
        cIdx      = zeros(Int64,N)
        for (i,x) in enumerate(eachrow(X))
            cost = Inf
            for (j,z) in enumerate(eachrow(Z))
               if (norm(x-z)^2  < cost)
                   cost    =  norm(x-z)^2
                   cIdx[i] = j
               end
            end
        end

        # Determining the new representative by each cluster
        #for (j,z) in enumerate(eachrow(Z))
        for j in  1:K
            Cⱼ = X[cIdx .== j,:] # Selecting the constituency by boolean selection
            Z[j,:] = sum(Cⱼ,dims=1) ./ size(Cⱼ)[1]
        end

        # Checking termination condition: clusters didn't move any more
        if cIdx == cIdx_prev
            return (cIdx,Z)
        else
            cIdx_prev = cIdx
        end

    end
end

kmeans

In [9]:
(clIdx,Z) = kmeans([1 10.5;1.5 10.8; 1.8 8; 1.7 15; 3.2 40; 3.6 32; 3.3 38; 5.1 -2.3; 5.2 -2.4],3)

([2, 2, 2, 2, 3, 3, 3, 1, 1], [5.15 -2.3499999999999996; 1.5 11.075; 3.366666666666667 36.666666666666664])

In [10]:
# Basic K-Medoids Algorithm (Lecture/segment 14.3 of https://www.edx.org/course/machine-learning-with-python-from-linear-models-to)

"""Square Euclidean distance"""
square_euclidean(x,y) = norm(x-y)^2

"""Cosine distance"""
cos_distance(x,y) = dot(x,y)/(norm(x)*norm(y))

cos_distance

In [11]:
"""
  kmedoids(X,K;dist,initStrategy,Z₀)

Compute K-Medoids algorithm to identify K clusters of X using distance definition `dist`

# Parameters:
* `X`: a (n x d) data to clusterise
* `K`: Number of cluster wonted
* `dist`: Function to employ as distance (must accept two vectors). Default to squared Euclidean.
* `initStrategy`: Wheter to select the initial representative vectors:
  * `random`: randomly in the X space
  * `grid`: using a grid approach
  * `shuffle`: selecting randomly within the available points [default]
  * `given`: using a provided set of initial representatives provided in the `Z₀` parameter
 * `Z₀`: Provided (K x D) matrix of initial representatives (used only together with the `given` initStrategy) [default: `nothing`]

# Returns:
* A tuple of two items, the first one being a vector of size N of ids of the clusters associated to each point and the second one the (K x D) matrix of representatives

# Notes:
* Some returned clusters could be empty

# Example:
```julia
julia> (clIdx,Z) = kmedoids([1 10.5;1.5 10.8; 1.8 8; 1.7 15; 3.2 40; 3.6 32; 3.3 38; 5.1 -2.3; 5.2 -2.4],3,dist = (x,y) -> norm(x-y)^2,initStrategy="grid")
```
"""
function kmedoids(X,K;dist=(x,y) -> norm(x-y)^2,initStrategy="shuffle",Z₀=nothing)
    (n,d) = size(X)
    # Random choice of initial representative vectors
    Z₀ = initRepresentatives(X,K,initStrategy=initStrategy,Z₀=Z₀)
    Z = Z₀
    cIdx_prev = zeros(Int64,n)

    # Looping
    while true
        # Determining the constituency of each cluster
        cIdx      = zeros(Int64,n)
        for (i,x) in enumerate(eachrow(X))
            cost = Inf
            for (j,z) in enumerate(eachrow(Z))
               if (dist(x,z) < cost)
                   cost =  dist(x,z)
                   cIdx[i] = j
               end
            end
        end

        # Determining the new representative by each cluster (within the points member)
        #for (j,z) in enumerate(eachrow(Z))
        for j in  1:K
            Cⱼ = X[cIdx .== j,:] # Selecting the constituency by boolean selection
            nⱼ = size(Cⱼ)[1]     # Size of the cluster
            if nⱼ == 0 continue end # empty continuency. Let's not do anything. Stil in the next batch other representatives could move away and points could enter this cluster
            bestCost = Inf
            bestCIdx = 0
            for cIdx in 1:nⱼ      # candidate index
                 candidateCost = 0.0
                 for tIdx in 1:nⱼ # target index
                     candidateCost += dist(Cⱼ[cIdx,:],Cⱼ[tIdx,:])
                 end
                 if candidateCost < bestCost
                     bestCost = candidateCost
                     bestCIdx = cIdx
                 end
            end
            Z[j,:] = reshape(Cⱼ[bestCIdx,:],1,d)
        end

        # Checking termination condition: clusters didn't move any more
        if cIdx == cIdx_prev
            return (cIdx,Z)
        else
            cIdx_prev = cIdx
        end

    end
end

kmedoids

In [12]:
(clIdx,Z) = kmedoids([1 10.5;1.5 10.8; 1.8 8; 1.7 15; 3.2 40; 3.6 32; 3.3 38; 5.1 -2.3; 5.2 -2.4],3,dist = (x,y) -> norm(x-y)^2,initStrategy="grid")

([2, 2, 2, 2, 3, 3, 3, 1, 1], [5.1 -2.3; 1.5 10.8; 3.3 38.0])

In [13]:
# The EM algorithm (Lecture/segment 16.5 of https://www.edx.org/course/machine-learning-with-python-from-linear-models-to)

""" PDF of a multidimensional normal with no covariance and shared variance across dimensions"""
normalFixedSd(x,μ,σ²) = (1/(2π*σ²)^(length(x)/2)) * exp(-1/(2σ²)*norm(x-μ)^2)

normalFixedSd

In [17]:
# The E-M Algorithm
"""
  em(X,K;p₀,μ₀,σ²₀,tol)

Compute Expectation-Maximisation algorithm to identify K clusters of X data assuming a Gaussian Mixture probabilistic Model.

# Parameters:
* `X`  :      A (n x d) data to clusterise
* `K`  :      Number of cluster wanted
* `p₀` :      Initial probabilities of the categorical distribution (K x 1) [default: `nothing`]
* `μ₀` :      Initial means (K x d) of the Gaussian [default: `nothing`]
* `σ²₀`:      Initial variance of the gaussian (K x 1). We assume here that the gaussian has the same variance across all the dimensions [default: `nothing`]
* `tol`:      Initial tolerance to stop the algorithm [default: 0.0001]
* `msgStep` : Iterations between update messages. Use 0 for no updates [default: 10]

# Returns:
* A named touple of:
  * `pⱼₓ`: Matrix of size (N x K) of the probabilities of each point i to belong to cluster j
  * `p`  : Probabilities of the categorical distribution (K x 1)
  * `μ`  : Means (K x d) of the Gaussian
  * `σ²` : Variance of the gaussian (K x 1). We assume here that the gaussian has the same variance across all the dimensions
  * `ϵ`  : Vector of the discrepancy (matrix norm) between pⱼₓ and the lagged pⱼₓ at each iteration

# Example:
```julia
julia> clusters = em([1 10.5;1.5 10.8; 1.8 8; 1.7 15; 3.2 40; 3.6 32; 3.3 38; 5.1 -2.3; 5.2 -2.4],3,msgStep=1)
```
"""
function em(X,K;p₀=nothing,μ₀=nothing,σ²₀=nothing,tol=0.0001,msgStep=10)
    # debug:
    #X = [1 10.5;1.5 10.8; 1.8 8; 1.7 15; 3.2 40; 3.6 32; 3.3 38; 5.1 -2.3; 5.2 -2.4]
    #K = 3
    #p₀=nothing; μ₀=nothing; σ²₀=nothing; tol=0.0001
    (N,D) = size(X)

    # Initialisation of the parameters if not provided
    minX = minimum(X,dims=1)
    maxX = maximum(X,dims=1)
    varX = mean(var(X,dims=1))/K^2
    pⱼ = isnothing(p₀) ? fill(1/K,K) : p₀
    if !isnothing(μ₀)
        μ = μ₀
    else
        μ = zeros(Float64,K,D)
        for d in 1:D
                μ[:,d] = collect(range(minX[d], stop=maxX[d], length=K))
        end
    end
    σ² = isnothing(σ²₀) ? fill(varX,K) : σ²₀
    pⱼₓ = zeros(Float64,N,K)

    ϵ = Float64[]
    while(true)
        # E Step: assigning the posterior prob p(j|xi)
        pⱼₓlagged = copy(pⱼₓ)
        for n in 1:N
            px = sum([pⱼ[j]*normalFixedSd(X[n,:],μ[j,:],σ²[j]) for j in 1:K])
            for k in 1:K
                pⱼₓ[n,k] = pⱼ[k]*normalFixedSd(X[n,:],μ[k,:],σ²[k])/px
            end
        end
        # M step: find parameters that maximise the likelihood
        nⱼ = sum(pⱼₓ,dims=1)'
        n = sum(nⱼ)
        pⱼ = nⱼ ./ n
        μ = (pⱼₓ' * X) ./ nⱼ
        σ² = [sum([pⱼₓ[n,j] * norm(X[n,:]-μ[j,:])^2 for n in 1:N]) for j in 1:K ] ./ (nⱼ .* D)

        push!(ϵ,norm(pⱼₓlagged - pⱼₓ))

        if msgStep != 0 && (length(ϵ) % msgStep == 0 || length(ϵ) == 1)
           println("Iter. $(length(ϵ))\t: $(ϵ[end])")
        end
        if (ϵ[end] < tol)
            return (pⱼₓ=pⱼₓ, pⱼ=pⱼ,μ=μ,σ²=σ²,ϵ=ϵ)
        end
    end
end

em

In [19]:
clusters = em([1 10.5;1.5 10.8; 1.8 8; 1.7 15; 3.2 40; 3.6 32; 3.3 38; 5.1 -2.3; 5.2 -2.4],3,msgStep=1)
clusters.pⱼₓ

Iter. 1	: 2.887117418966972
Iter. 2	: 0.7123655504946171
Iter. 3	: 0.11165110966805576
Iter. 4	: 3.794945858137016e-10


9×3 Array{Float64,2}:
 0.0  1.0          6.00753e-27
 0.0  1.0          2.5648e-26 
 0.0  1.0          2.31584e-31
 0.0  1.0          9.14102e-18
 0.0  8.60227e-57  1.0        
 0.0  1.7555e-29   1.0        
 0.0  1.33625e-49  1.0        
 1.0  4.29262e-16  1.61741e-60
 1.0  2.52008e-16  7.99659e-61